# Lego Project
### "Exploring The Evolution Of Lego"

#### Import libraries and datasets

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import plotly.express as px 

In [2]:
# We get present working directory in order to have an independent notebook
pwd = os.getcwd()

In [3]:
# Reading data
ls = pd.read_csv(pwd + '\\lego_sets.csv')
pt = pd.read_csv(pwd + '\\parent_themes.csv')

In [4]:
pd.set_option('display.max_rows', 200)

In [ ]:
# First touch with dataset
ls.head()

In [ ]:
pt.head()

In [ ]:
# We observe that there is an anomaly. So we guess, that there are null values
ls.count()

In [ ]:
# No null values
pt.count()

#### Fix missing values

In [ ]:
# Check for null values
for col in ls.columns:
    missing_percentage = round(np.mean(ls[col].isnull()), 2)
    print(f'{col} - {missing_percentage} %')

In [10]:
# Null values for num_parts are acceptable, depending on the problem
ls.dropna(subset=['set_num'], inplace=True)

In [ ]:
# No null values anymore
ls.isnull().sum()

#### What percentage of all licensed sets ever released, were Star Wars themed ?

In [ ]:
# Join lego_sets table with parent_themes table
mer = ls.merge(pt, left_on='parent_theme', right_on='name')
mer.drop(columns='name_y', inplace=True)
mer.rename(columns={'name_x':'name'}, inplace=True)
mer.head()

In [ ]:
# First we find all the licensed set ever released
lic = mer.loc[mer['is_licensed'] == True]
lic

In [ ]:
# Now we isolate the star wars parent theme
star_wars = lic.loc[lic['parent_theme'] == 'Star Wars']
star_wars

In [ ]:
# Now it's time for the percentage of star wars
the_force = int(star_wars.shape[0]/lic.shape[0]*100)
print(f'The percentage of all Star Wars themed licensed sets ever releashed is: {the_force} %')

#### In which year was Star Wars not the most popular licensed theme ?

In [ ]:
# This gives me the year with the minimum number of sets, in Star Wars lego history
# This is not what we search for, but we will keep this. Maybe it is usefull to know that
yearly_selling_sets = star_wars.groupby('year')['set_num'].count()
sorted_years = yearly_selling_sets.sort_values(ascending=True)
worst_selling_year = sorted_years.iloc[0:1]
worst_selling_year

In [ ]:
# We isolate the data we need
yearly_theme_num_of_sets = lic.groupby(['year', 'parent_theme'])['set_num'].count().reset_index()
yearly_theme_num_of_sets.rename(columns={'set_num':'sets_count'}, inplace=True)
yearly_theme_num_of_sets

In [ ]:
# We take the maximum from each year, in order to see year by year the winning theme
each_year_max = yearly_theme_num_of_sets.sort_values('sets_count', ascending=False).drop_duplicates(['year'])
each_year_max.sort_values('year', inplace=True)
each_year_max.reset_index(inplace=True)
each_year_max.drop(columns=['index'], inplace=True)
each_year_max

In [ ]:
new_era = each_year_max.iloc[18,0]
new_era
print(f'The year that Star Wars was not the most popular licensed theme was: {new_era}')

#### How many unique sets were released each year (1950-2017) ?

In [ ]:
# Summary of all sets year by year, from the begging of Lego History
set_per_year = mer.groupby('year')['set_num'].count().reset_index()
set_per_year

In [21]:
# Line plot with plotly
px.line(set_per_year, x='year', y='set_num', title='Lego Sets (1950-2017)', markers=True)

#### Which is the contribution (%) of all parent themes in the most successfull year

In [22]:
# Find id for year with maximum number of sets and the exect year
best_year_id = set_per_year['set_num'].idxmax()
best_year = set_per_year.iloc[best_year_id,0]
best_num_of_sets = set_per_year.iloc[best_year_id,1]

In [23]:
# We prepare the dataframe depending on year and parent_theme
temp = mer.groupby(['year', 'parent_theme'])['set_num'].count().reset_index()

In [ ]:
# We pick the frame with the most successfull year
best_year_df = temp.loc[temp['year'] == best_year].reset_index()
best_year_df.drop(columns='index', inplace=True)
best_year_df

In [ ]:
# Parent themes contribution %
best_year_df['contr %'] = best_year_df.iloc[:,2]/best_num_of_sets*100
best_year_df['contr %'] = best_year_df['contr %'].apply(lambda x: round(x,2))
best_year_df.sort_values(['contr %', 'parent_theme'], inplace=True)
best_year_df

In [62]:
px.bar(best_year_df, x='contr %', y='parent_theme', color='set_num', color_continuous_scale='earth_r', orientation='h', height=900, title='Parent themes contribution in the most successfull year (2014)')